In [1]:
from attention_dynamic_model import AttentionDynamicModel
import tensorflow as tf
import time
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2022-10-24 17:27:16.584715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1
env: TF_GPU_ALLOCATOR=cuda_malloc_async


2022-10-24 17:27:18.041874: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 17:27:18.072034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 17:27:18.072207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Benchmarking AM-D
=================

This experiment will train an AM-D model based on the following hyperparameters:

| Hyperparameter | Values |
| ---: | :--- |
| Learning Rate | 1e-4, 5e-5 1e-5 |
| Embedding Dimensions | 64, 128, 256 |
| Attention Heads | 6, 8, 10 |
| Encoder Layers | 1, 2, 3, 4 |



In [ ]:
learn_rates = [1e-4, 5e-5, 1e-5]
enc_dims    = [64, 128, 256]
attn_heads  = [6, 8, 10]
enc_layers  = [1, 2, 3, 4]
iterations  = [2, 2, 4, 8, 16, 32, 64, 128]

''' Constant Parameters '''
# AM-D Constants
tanh_clipping = 10
# Optimizer Constants (Adam)
beta_1        = 0.9
beta_2        = 0.999
epsilon       = 1e-07
amsgrad       = False
name          = "Adam"
# Environment Constants
graph_size         = 20
# Rollout Baseline Constants
wp_n_epochs        = 5
epoch              = 0
num_samples        = 10_000 #???
warmup_exp_beta    = 0.8
# Training Constants
samples            = 50_000 #1_280_000 # 512
batch              = 64
val_batch_size     = 1_000
start_epoch        = 0
grad_norm_clipping = 1.0
batch_verbose      = 1_000





for lr in learn_rates:
    for enc_dim in enc_dims:
        for attn_h in attn_heads:
            for enc_layer in enc_layers:
                # Build AM-D Model
                model_amd = AttentionDynamicModel(
                    embedding_dim  =enc_dim,
                    n_encode_layers=enc_layer,
                    n_heads        =attn_h,
                    tanh_clipping  =tanh_clipping
                )
                model_amd.set_decode_type('sampling')
                
                # Create Optimizer
                optimizer = Adam(
                    learning_rate=lr,
                    beta_1=beta_1,
                    beta_2=beta_2,
                    epsilon=epsilon,
                    amsgrad=amsgrad,
                    name=name,
                )
                
                # Baseline model
                baseline = RolloutBaseline(
                    model             = model_amd,
                    filename          = 'VRP_{}_{}'.format(graph_size, strftime("%Y-%m-%d", gmtime())),
                    from_checkpoint   = False,
                    path_to_checkpoint= None,
                    wp_n_epochs       = wp_n_epochs,
                    epoch             = epoch,
                    num_samples       = num_samples,
                    embedding_dim     = enc_dim,
                    graph_size        = graph_size
                    )
                for _iter in iterations:
                    date = 'oct_24'
                    filename = f'checkpoints/AMD-banchmarking-trained_on-{_iter * num_samples}-num_layers-{enc_layer}-attn-{attn_h}-embedding_dim-{enc_dim}-lr-{lr}-date-{date}.ckp'
                    validation_dataset = create_data_on_disk(
                        graph_size     =graph_size,
                        num_samples    =num_samples,
                        is_save        =False,
                        filename       =None,
                        is_return      =True,
                        seed           = 42
                    )
                    train_model(
                        optimizer          = optimizer,
                        model_tf           = model_amd,
                        baseline           = baseline,
                        validation_dataset = validation_dataset,
                        samples            = samples,
                        batch              = batch,
                        val_batch_size     = val_batch_size,
                        start_epoch        = start_epoch,
                        end_epoch          = _iter,
                        from_checkpoint    = False,
                        grad_norm_clipping = grad_norm_clipping,
                        batch_verbose      = batch_verbose,
                        graph_size         = graph_size,
                        filename           = None
                        )
                    model_amd.save_weights(filename)
                    # To Do: Add information about progress